In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Heart_rate/GSE12385'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression changes in Peripheral Blood Mononuclear cells (PBMC) induced by physical activity"
!Series_summary	"Gene expression changes in Peripheral Blood Mononuclear cells (PBMC) induced by physical activity was investigated in  sedentary middle-aged men (mean age 52.6 years and BMI 29.1) who undertook a 24-week physical activity programme with blood sampling in the pre-exercise period ,  at the end of 24-weeks prescribed physical activity , and following a two-week detraining period."
!Series_overall_design	"AgilentTM Whole Human Genome Oligo Microarrays were utilised to examine the effects of physical activity on mRNA expression profiles of the Peripheral Blood Mononuclear cells (PBMC) at 3 time points (pre-exercise, after 24 weeks physical activity, and at 26 weeks after 2 weeks detraining. There were 12 participants in this programme."
Sample Characteristics Dictionary:
{0: ['Gender: Male'], 1: ['Age (years): 48', 'Age (years): 54', 'Age

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
if "AgilentTM Whole Human Genome Oligo Microarrays" in "!Series_overall_design":
    is_gene_available = True

# Determine the keys for 'Heart_rate', 'age', and 'gender'
sample_characteristics_dict = {
    0: ['Gender: Male'], 
    1: ['Age (years): 48', 'Age (years): 54', 'Age (years): 56', 'Age (years): 53', 'Age (years): 62', 'Age (years): 59', 'Age (years): 46', 'Age (years): 50', 'Age (years): 51']
    #... other entries...
}

# Heart_rate is not explicitly mentioned in the Sample Characteristics Dictionary
trait_row = None

# Age key (identifying the appropriate key in the dictionary)
for key, values in sample_characteristics_dict.items():
    if any("Age" in value for value in values):
        age_row = key
        break

# Gender key (identifying the appropriate key in the dictionary)
for key, values in sample_characteristics_dict.items():
    if any("Gender" in value for value in values):
        gender_row = key
        break

# Define conversion functions
def convert_trait(value):
    return None  # Since Heart_rate data is not available

def convert_age(value):
    try:
        # Extract value after ':'
        return int(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    try:
        # Extract value after ':'
        gender = value.split(': ')[1].strip().lower()
        return 1 if gender == 'male' else 0 if gender == 'female' else None
    except (ValueError, IndexError):
        return None

# Save metadata
save_cohort_info('GSE12385', './preprocessed/Heart_rate/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Heart_rate', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Heart_rate/trait_data/GSE12385.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
else:
    print("No clinical feature extraction due to the absence of Heart_rate data.")


No clinical feature extraction due to the absence of Heart_rate data.
